### Расчет метрик эффективности для рекомендаций

В условиях отсутствия повторного взаимодействия между командами, кейсами и участниками оценка качества рекомендаций может быть основана на исторических данных. Для этого используются метрики, которые сопоставляют рекомендации с фактическими данными. Ниже приведены подходы к расчету метрик для каждого типа рекомендаций:

---

#### **1. Команда-Кейс: подбор кейса для команды**

Метрики:
- **Hit Rate@N**:
  - **Описание**: Доля случаев, когда правильный кейс оказался в топ-N рекомендаций.
- **NDCG@N**:
  - **Описание**: Учитывает позиции правильных кейсов в списке рекомендаций, придавая больший вес верхним позициям.
- **Coverage**:
  - **Описание**: Доля кейсов, которые были рекомендованы хотя бы одной команде.

---

#### **2. Кейс-Команда: подбор команды для кейса**

Метрики:
- **Hit Rate@N**:
  - **Описание**: Доля случаев, когда подходящая команда оказалась в топ-N рекомендаций.
- **NDCG@N**:
  - **Описание**: Учитывает позиции правильных команд в списке рекомендаций, придавая больший вес верхним позициям.
- **Coverage**:
  - **Описание**: Доля команд, которые были рекомендованы хотя бы для одного кейса.

---

#### **3. Человек-Команда: подбор команды для участника**

Метрики:
- **Hit Rate@N**:
  - **Описание**: Проверяет, попала ли команда, в которую входит участник, в топ-N рекомендаций.
- **NDCG@N**:
  - **Описание**: Учитывает позиции правильной команды в списке рекомендаций, придавая больший вес верхним позициям.
- **Coverage**:
  - **Описание**: Доля команд, которые были предложены хотя бы одному участнику.

---

### **A/B-тестирование**

Для проведения A/B-тестов в системе рекомендаций, даже при отсутствии повторных взаимодействий, можно использовать метрики для сравнения качества рекомендаций между тестовой и контрольной группами.

#### **Общие метрики для A/B-тестирования**
1. **Hit Rate@N**:
   - **Цель**: Сравнить долю случаев, когда правильные кейсы, команды или участники оказались в топ-N рекомендациях, между тестовой и контрольной группами.

2. **NDCG@N**:
   - **Цель**: Измерить качество ранжирования рекомендаций, уделяя внимание позициям релевантных объектов в списке.

3. **Coverage**:
   - **Цель**: Проверить, насколько разнообразны рекомендации в разных группах.


In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Content-based classification

In [166]:
df = pd.read_csv('../data/historical_data_with_roles.csv')

In [167]:
df.head()

,team_id,team_title,case_title,case_description,user_fio,person_skills,case_required_roles
0,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Додин Даниил Дмитриевич,NaN,"ML engineer, Python Backend, DevOps, CV engine..."
1,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Хижний Никита Александрович,"Data Science, Computer Vision, Confluence, Dat...","ML engineer, Python Backend, DevOps, CV engine..."
2,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Папуашвили Георгий Давидович,"С++, AirFlow, Computer Vision, Confluence, Dat...","ML engineer, Python Backend, DevOps, CV engine..."
3,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Коробов Максим Евгеньевич,"С++, Computer Vision, Excel, Git, Go, Jupyter,...","ML engineer, Python Backend, DevOps, CV engine..."
4,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Назербаев Руслан Бахтиярович,"Back-end разработка, С, CSS, Django, Front-end...","ML engineer, Python Backend, DevOps, CV engine..."


In [168]:
available_user_skills = pd.read_csv('../data/user_skills.csv')['Skills'].tolist()

# Function to filter person_skills
def filter_skills(skills):
    if not isinstance(skills, str):  # Handle NaN or non-string values
        return ""
    filtered = [skill for skill in skills.split(", ") if skill in available_user_skills]
    return ", ".join(filtered)

# Apply the filter to the person_skills column
df["person_skills"] = df["person_skills"].apply(filter_skills)

In [169]:
role_to_skills_mapping = {
    "Java Backend": [
        "Java", "Spring Boot", "PostgreSQL", "Git", "Построение Rest API", "Умение работать с API",
        "Back-end разработка", "Linux", "Docker", "Kubernetes", "SQL"
    ],
    "C# Backend": [
        "C#", "Back-end разработка", "Docker", "SQL", "Git", "Умение работать с API", "СУБД PostgreSQL",
        ".NET", "Linux", "Kubernetes", "Nginx", "Управление проектами"
    ],
    "Go Backend": [
        "Go", "Back-end разработка", "Docker", "Git", "Kubernetes", "SQL", "Linux", "Helm", "Nginx",
        "Умение работать с API", "Grafana", "Управление проектами"
    ],
    "Python Backend": [
        "Python", "Django", "Docker", "Построение Rest API", "SQL", "СУБД PostgreSQL", "Git",
        "Back-end разработка", "Linux", "Kubernetes", "Nginx", "Умение работать с API"
    ],
    "C++ Backend": [
        "C++", "Back-end разработка", "Git", "Linux", "Docker", "Kubernetes", "SQL", "Умение работать с API",
        "PostgreSQL", "Управление проектами"
    ],
    "Frontend": [
        "React", "CSS", "HTML", "JavaScript", "Tailwind", "Next", "Vue", "Git", "Figma", "Canva",
        "UI/UX", "Zustand", "SSR", "MUI", "Shadcn", "Nginx"
    ],
    "ML engineer": [
        "Machine Learning", "Python", "TensorFlow", "Scikit-Learn", "Pandas", "NumPy", "Deep Learning",
        "PyTorch", "Data Science", "SQL", "Matplotlib", "Seaborn", "Управление проектами", "Jupyter",
        "Keras", "OpenCV", "Computer Vision"
    ],
    "DevOps": [
        "Docker", "Kubernetes", "Linux", "Helm", "Nginx", "Grafana", "AirFlow", "K8S", "Git",
        "Nexus", "ELK", "CDN", "S3", "Hadoop", "Управление проектами", "Умение работать с API"
    ],
    "Тестировщик": [
        "Python", "PostgreSQL", "Умение работать с API", "Atlassian stack [Jira, Confluence]", "Linux",
        "SQL", "Git", "Управление проектами", "Docker", "Automated testing"
    ],
    "Аналитик": [
        "Data Science", "SQL", "Pandas", "Математическая статистика", "Управление проектами",
        "Data Engineering", "Разработка моделей данных", "Python", "Jupyter", "Умение работать с API"
    ],
    "Дизайнер": [
        "Figma", "Canva", "CSS", "UI/UX", "Photoshop", "Adobe XD", "JavaScript", "HTML", "Tailwind", "React"
    ],
    "Инженер БПЛА": [
        "C++", "Python", "Computer Vision", "Linux", "Kubernetes", "Docker", "OpenCV", "TensorFlow",
        "Machine Learning", "Git", "ROS"
    ],
    "CV engineer": [
        "Computer Vision", "OpenCV", "Python", "PyTorch", "TensorFlow", "Machine Learning", "Deep Learning",
        "Scikit-Learn", "NumPy", "Pandas", "Data Science", "Jupyter", "Keras", "ONNX Runtime"
    ],
    "ML Ops Engineer": [
        "DevOps", "AirFlow", "Kubernetes", "ONNX Runtime", "Docker", "Linux", "Grafana", "TensorFlow",
        "Machine Learning", "PyTorch", "Git", "Hadoop", "Nginx", "K8S", "S3", "CDN", "Управление проектами"
    ]
}

def map_roles_to_skills(required_roles, mapping):
    roles = required_roles.split(", ")
    skills = set()
    for role in roles:
        skills.update(mapping.get(role, []))
    return list(skills)

# Apply mapping to create the 'case_required_skills' column
df['case_required_skills'] = df['case_required_roles'].apply(lambda x: map_roles_to_skills(x, role_to_skills_mapping))

In [170]:
df.head()

,team_id,team_title,case_title,case_description,user_fio,person_skills,case_required_roles,case_required_skills
0,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Додин Даниил Дмитриевич,,"ML engineer, Python Backend, DevOps, CV engine...","[Linux, Умение работать с API, PyTorch, Back-e..."
1,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Хижний Никита Александрович,"Data Science, Computer Vision, Data Engineerin...","ML engineer, Python Backend, DevOps, CV engine...","[Linux, Умение работать с API, PyTorch, Back-e..."
2,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Папуашвили Георгий Давидович,"AirFlow, Computer Vision, Data Engineering, Da...","ML engineer, Python Backend, DevOps, CV engine...","[Linux, Умение работать с API, PyTorch, Back-e..."
3,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Коробов Максим Евгеньевич,"Computer Vision, Git, Go, Jupyter, Machine Lea...","ML engineer, Python Backend, DevOps, CV engine...","[Linux, Умение работать с API, PyTorch, Back-e..."
4,81,10pandas,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Назербаев Руслан Бахтиярович,"Back-end разработка, CSS, Django, Git, HTML, L...","ML engineer, Python Backend, DevOps, CV engine...","[Linux, Умение работать с API, PyTorch, Back-e..."


In [171]:
df.drop(['team_title'], axis=1, inplace=True)

In [ ]:
df.to_csv('df.csv', index=False)

In [53]:
df.head()

,team_id,case_title,case_description,user_fio,person_skills,case_required_skills
0,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Додин Даниил Дмитриевич,,"[Linux, Умение работать с API, PyTorch, Back-e..."
1,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Хижний Никита Александрович,"Data Science, Computer Vision, Data Engineerin...","[Linux, Умение работать с API, PyTorch, Back-e..."
2,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Папуашвили Георгий Давидович,"AirFlow, Computer Vision, Data Engineering, Da...","[Linux, Умение работать с API, PyTorch, Back-e..."
3,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Коробов Максим Евгеньевич,"Computer Vision, Git, Go, Jupyter, Machine Lea...","[Linux, Умение работать с API, PyTorch, Back-e..."
4,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Назербаев Руслан Бахтиярович,"Back-end разработка, CSS, Django, Git, HTML, L...","[Linux, Умение работать с API, PyTorch, Back-e..."


## Команда - кейс (рекомендация кейсов для команды)

In [54]:
# Combine team information into one row per team
team_data = df.groupby('team_id').agg({
    'case_title': 'first',
    'case_description': 'first',
    'case_required_skills': 'first',
    'person_skills': lambda x: list(x.dropna())
}).reset_index()

# Create features by combining team skills and case description/required skills
def prepare_features(row):
    case_features = ' '.join(row['case_required_skills'])  # Combine required skills
    team_features = ' '.join(row['person_skills'])  # Combine team member skills
    return f"{case_features} {team_features}"

team_data['combined_features'] = team_data.apply(prepare_features, axis=1)

# Target variable: adjust based on your real data
team_data['target'] = 1  # Example target (all suitable)

In [ ]:
# Generate negative examples by selecting cases not chosen by the team
def generate_negative_examples(df):
    negative_samples = []
    unique_cases = df['case_title'].unique()
    
    for team_id, group in df.groupby('team_id'):
        chosen_cases = group['case_title'].unique()
        non_chosen_cases = set(unique_cases) - set(chosen_cases)
        
        for case in non_chosen_cases:
            case_info = df[df['case_title'] == case].iloc[0]
            team_skills = group['person_skills'].dropna().tolist()
            negative_samples.append({
                'team_id': team_id,
                'case_title': case_info['case_title'],
                'case_description': case_info['case_description'],
                'case_required_skills': case_info['case_required_skills'],
                'person_skills': team_skills,
                'target': 0  # Mark as a negative example
            })
    
    return pd.DataFrame(negative_samples)

# Generate negative examples
negative_examples = generate_negative_examples(df)

# Combine original dataset (positive examples) with negative examples
positive_examples = df.groupby('team_id').agg({
    'case_title': 'first',
    'case_description': 'first',
    'case_required_skills': 'first',
    'person_skills': lambda x: list(x.dropna())
}).reset_index()
positive_examples['target'] = 1

combined_data = pd.concat([positive_examples, negative_examples], ignore_index=True)

# Create combined features for the model
combined_data['combined_features'] = combined_data.apply(prepare_features, axis=1)

In [55]:
team_data.head()

,team_id,case_title,case_description,case_required_skills,person_skills,combined_features,target
0,1,Детектирование борщевика сосновского на основа...,Цель итоговой аттестационной работы – разработ...,"[Linux, UI/UX, Умение работать с API, PyTorch,...","[Python, Git, Python, , Canva, CSS, Data Scien...",Linux UI/UX Умение работать с API PyTorch Auto...,1
1,2,Платформа командообразования,Для эффективного командообразования студентов ...,"[UI/UX, Linux, Умение работать с API, Back-end...","[CSS, HTML, Git, Linux, Django, Docker, DevOps...",UI/UX Linux Умение работать с API Back-end раз...,1
2,3,Разработка системы ECM для авиационного двигателя,Имеется техническая документация по авиационны...,"[Linux, UI/UX, Умение работать с API, PyTorch,...","[Linux, Matplotlib, NumPy, Python, Умение рабо...",Linux UI/UX Умение работать с API PyTorch Back...,1
3,4,Развитие системы GlobalMonitoring в части адап...,"Компания АО ""Медиатек"" собирает данные с больш...","[Linux, Умение работать с API, Automated testi...","[, , , Back-end разработка, CSS, Docker, Figma...",Linux Умение работать с API Automated testing ...,1
4,6,Применение нейронных сетей для балансировки на...,На данный момент современные веб и обратные пр...,"[Linux, Умение работать с API, PyTorch, Automa...","[Linux, Git, Figma, Python, Git, Linux, Git, N...",Linux Умение работать с API PyTorch Automated ...,1


# Человек - команда (подбор команды)

In [96]:
df = pd.read_csv('data.csv')

In [151]:
df.shape

(1672, 6)

In [153]:
df.head()

,team_id,case_title,case_description,user_fio,person_skills,case_required_skills
0,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Додин Даниил Дмитриевич,,"[Linux, Умение работать с API, PyTorch, Back-e..."
1,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Хижний Никита Александрович,"Data Science, Computer Vision, Data Engineerin...","[Linux, Умение работать с API, PyTorch, Back-e..."
2,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Папуашвили Георгий Давидович,"AirFlow, Computer Vision, Data Engineering, Da...","[Linux, Умение работать с API, PyTorch, Back-e..."
3,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Коробов Максим Евгеньевич,"Computer Vision, Git, Go, Jupyter, Machine Lea...","[Linux, Умение работать с API, PyTorch, Back-e..."
4,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Назербаев Руслан Бахтиярович,"Back-end разработка, CSS, Django, Git, HTML, L...","[Linux, Умение работать с API, PyTorch, Back-e..."


In [172]:
df['person_skills'] = df['person_skills'].replace('', pd.NA)

In [173]:
df.isna().sum()

team_id                   0
case_title                0
case_description          0
user_fio                  8
person_skills           730
case_required_roles       0
case_required_skills      0
dtype: int64

In [174]:
df = df.dropna()

In [176]:
df['person_skills'] = df['person_skills'].tolist()

In [178]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import ast

In [179]:
# 1. Обработка текста с помощью TF-IDF
tfidf_vectorizer_desc = TfidfVectorizer(max_features=100)  # Ограничиваем количество фичей
tfidf_vectorizer_title = TfidfVectorizer(max_features=50)

# Обработка текстовых признаков
tfidf_case_desc = tfidf_vectorizer_desc.fit_transform(df['case_description'].fillna(""))
tfidf_case_title = tfidf_vectorizer_title.fit_transform(df['case_title'].fillna(""))

# Преобразуем в DataFrame
tfidf_case_desc_df = pd.DataFrame(tfidf_case_desc.toarray(), columns=[f'desc_tfidf_{i}' for i in range(tfidf_case_desc.shape[1])])
tfidf_case_title_df = pd.DataFrame(tfidf_case_title.toarray(), columns=[f'title_tfidf_{i}' for i in range(tfidf_case_title.shape[1])])

In [180]:
# 2. Преобразование навыков
mlb = MultiLabelBinarizer()

# Преобразуем строковые списки в настоящие списки
def safe_eval(x):
    if isinstance(x, list):  # Если это уже список
        return x
    if isinstance(x, str):  # Если это строка
        try:
            # Пробуем обработать строку как список
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            # Если строка не является валидным списком, разбиваем по запятой
            return [skill.strip() for skill in x.split(',')]
    return []  # Возвращаем пустой список для других типов данных


df['person_skills'] = df['person_skills'].apply(safe_eval)

# Группируем навыки по team_id
team_skills_mapping = (
    df.groupby('team_id')['person_skills']
    .apply(lambda skills: list(set(skill for sublist in skills for skill in sublist)))
    .to_dict()
)

# Добавляем колонку team_skills, используя mapping
df['team_skills'] = df['team_id'].map(team_skills_mapping)

df['person_skills'] = df['person_skills'].apply(safe_eval)
df['team_skills'] = df['team_skills'].apply(safe_eval)
df['case_required_skills'] = df['case_required_skills'].apply(safe_eval)

# Объединение всех навыков для создания общего набора уникальных навыков
all_skills = set().union(*df['person_skills'], *df['team_skills'], *df['case_required_skills'])

# Бинарное кодирование навыков
person_skills_encoded = pd.DataFrame(mlb.fit_transform(df['person_skills']), columns=[f'person_skill_{skill}' for skill in mlb.classes_])
team_skills_encoded = pd.DataFrame(mlb.fit_transform(df['team_skills']), columns=[f'team_skill_{skill}' for skill in mlb.classes_])
case_skills_encoded = pd.DataFrame(mlb.fit_transform(df['case_required_skills']), columns=[f'case_skill_{skill}' for skill in mlb.classes_])

In [181]:
# 3. Объединение всех данных в единый DataFrame
processed_data = pd.concat([df.reset_index(drop=True), 
                            tfidf_case_desc_df, 
                            tfidf_case_title_df, 
                            person_skills_encoded, 
                            team_skills_encoded, 
                            case_skills_encoded], axis=1)

# Удаляем оригинальные текстовые и строковые столбцы
# processed_data.drop(['case_description', 'case_title', 'person_skills', 'team_skills', 'case_required_skills'], axis=1, inplace=True)

In [182]:
processed_data.shape

(942, 313)

In [184]:
import random
positive_examples = processed_data.copy()
positive_examples["target"] = 1

# Генерация отрицательных примеров
negative_examples = []
teams = processed_data["team_id"].unique()

for _, row in processed_data.iterrows():
    # Генерируем случайную команду, которая не совпадает с текущей
    negative_team = random.choice([team for team in teams if team != row["team_id"]])
    negative_example = row.copy()
    negative_example["team_id"] = negative_team
    
    # Генерируем новые team_skills для негативного примера
    negative_example['team_skills'] = processed_data[processed_data['team_id'] == negative_team]['team_skills'].iloc[0]
    
    negative_example["target"] = 0
    negative_examples.append(negative_example)

# Создание DataFrame с отрицательными примерами
negative_examples = pd.DataFrame(negative_examples)

# Объединение положительных и отрицательных примеров
dataset = pd.concat([positive_examples, negative_examples], ignore_index=True)

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset["target"])

# Сохраняем дополнительные данные для API
test_data_with_skills = test_data[["team_id", "person_skills", "team_skills", "case_required_roles", "target"]]
test_data_with_skills.to_csv("test_dataset_skills.csv", index=False)

In [185]:
train_data.drop(['case_description', 'case_title', 'person_skills', 'team_skills','case_required_roles', 'case_required_skills'], axis=1, inplace=True)
test_data.drop(['case_description', 'case_title', 'person_skills', 'team_skills','case_required_roles', 'case_required_skills'], axis=1, inplace=True)

In [186]:
dataset.head()

,team_id,case_title,case_description,user_fio,person_skills,case_required_roles,case_required_skills,team_skills,desc_tfidf_0,desc_tfidf_1,...,case_skill_UI/UX,case_skill_Vue,case_skill_Zustand,case_skill_Математическая статистика,case_skill_Построение Rest API,case_skill_Разработка моделей данных,case_skill_СУБД PostgreSQL,case_skill_Умение работать с API,case_skill_Управление проектами,target
0,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Хижний Никита Александрович,"[Data Science, Computer Vision, Data Engineeri...","ML engineer, Python Backend, DevOps, CV engine...","[Linux, Умение работать с API, PyTorch, Back-e...","[DevOps, Linux, Умение работать с API, PyTorch...",0.0,0.0,...,0,0,0,1,1,1,1,1,1,1
1,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Папуашвили Георгий Давидович,"[AirFlow, Computer Vision, Data Engineering, D...","ML engineer, Python Backend, DevOps, CV engine...","[Linux, Умение работать с API, PyTorch, Back-e...","[DevOps, Linux, Умение работать с API, PyTorch...",0.0,0.0,...,0,0,0,1,1,1,1,1,1,1
2,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Коробов Максим Евгеньевич,"[Computer Vision, Git, Go, Jupyter, Machine Le...","ML engineer, Python Backend, DevOps, CV engine...","[Linux, Умение работать с API, PyTorch, Back-e...","[DevOps, Linux, Умение работать с API, PyTorch...",0.0,0.0,...,0,0,0,1,1,1,1,1,1,1
3,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Назербаев Руслан Бахтиярович,"[Back-end разработка, CSS, Django, Git, HTML, ...","ML engineer, Python Backend, DevOps, CV engine...","[Linux, Умение работать с API, PyTorch, Back-e...","[DevOps, Linux, Умение работать с API, PyTorch...",0.0,0.0,...,0,0,0,1,1,1,1,1,1,1
4,81,Мультимодальный поиск по изображениям,"Проект ""Мультимодальный поиск по изображениям""...",Семенцов Олег Викторович,"[Data Science, Git, Jupyter, Matplotlib, NumPy...","ML engineer, Python Backend, DevOps, CV engine...","[Linux, Умение работать с API, PyTorch, Back-e...","[DevOps, Linux, Умение работать с API, PyTorch...",0.0,0.0,...,0,0,0,1,1,1,1,1,1,1


In [187]:
train_data.drop(['team_id', 'user_fio'], axis=1, inplace=True)
test_data.drop(['team_id', 'user_fio'], axis=1, inplace=True)

In [188]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset["target"])

test_data.to_csv("test_dataset.csv", index=False)

# Разделение на фичи и целевую переменную
X_train = train_data.drop(columns=["target"])
y_train = train_data["target"]

X_test = test_data.drop(columns=["target"])
y_test = test_data["target"]

# Указание текстовых и категориальных признаков
# Предположим, что 'desc_tfidf_...' и 'title_tfidf_...' — числовые признаки, а остальные — категориальные
# text_features = [col for col in X_train.columns if col.startswith("desc_tfidf_") or col.startswith("title_tfidf_")]
cat_features = []  # Если есть категориальные признаки, добавьте их сюда

# Создание пулов для CatBoost
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)

# Обучение CatBoostClassifier
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=10,
    verbose=100
)

model.fit(train_pool, eval_set=test_pool, verbose=100, early_stopping_rounds=50)

# Предсказания
y_pred = model.predict(test_pool)
y_pred_proba = model.predict_proba(test_pool)[:, 1]

# Оценка модели
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_proba))

0:	learn: 0.6923996	test: 0.6963377	best: 0.6963377 (0)	total: 29.5ms	remaining: 29.5s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6963377405
bestIteration = 0

Shrink model to first 1 iterations.
Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.58      0.49       189
           1       0.31      0.19      0.23       188

    accuracy                           0.38       377
   macro avg       0.36      0.38      0.36       377
weighted avg       0.36      0.38      0.36       377

ROC-AUC Score: 0.3341635708656986


In [215]:
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from sklearn.metrics import ndcg_score, average_precision_score
import numpy as np

# Предсказания
y_pred = model.predict(test_pool)
y_pred_proba = model.predict_proba(test_pool)[:, 1]

# F1-score
f1 = f1_score(y_test, y_pred)

# NDCG
# NDCG предполагает, что у нас есть вероятность (y_pred_proba) как релевантность
ndcg = ndcg_score([y_test], [y_pred_proba])

# MAP
# Mean Average Precision также рассчитывается на основе вероятностей
map_score = average_precision_score(y_test, y_pred_proba)

# Вывод метрик
print("F1-Score:", f1)
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_proba))
print("NDCG Score:", ndcg)
print("MAP Score:", map_score)

top_n = 5

# Сортировка по вероятностям и выбор топ-N
sorted_indices = np.argsort(-y_pred_proba)[:top_n]
y_test_top_n = np.array(y_test)[sorted_indices]
y_pred_proba_top_n = np.array(y_pred_proba)[sorted_indices]

# Перерасчет метрик для топ-N
ndcg_top_n = ndcg_score([y_test_top_n], [y_pred_proba_top_n])
map_top_n = average_precision_score(y_test_top_n, y_pred_proba_top_n)

print("NDCG (Top-N):", ndcg_top_n)
print("MAP (Top-N):", map_top_n)

# Полный отчет по классификации
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

F1-Score: 0.23178807947019867
ROC-AUC Score: 0.3341635708656986
NDCG Score: 0.7737802751563536
MAP Score: 0.39970522166979244
NDCG (Top-N): 0.5563625548325833
MAP (Top-N): 0.325

Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.58      0.49       189
           1       0.31      0.19      0.23       188

    accuracy                           0.38       377
   macro avg       0.36      0.38      0.36       377
weighted avg       0.36      0.38      0.36       377



In [239]:
import requests
import pandas as pd
from sklearn.metrics import roc_auc_score, f1_score, ndcg_score, average_precision_score
import numpy as np

# URL вашего запущенного FastAPI приложения
API_URL = "http://127.0.0.1:8000/recommend_team_to_person"

# Функция для тестирования API и расчета метрик
def test_api_and_evaluate(api_url, test_data, model, top_n=5):
    """
    Тестирует API FastAPI для рекомендаций и рассчитывает метрики.

    Args:
        api_url (str): URL эндпоинта API.
        test_data (DataFrame): Тестовый набор данных.
        model: Обученная модель CatBoost.
        top_n (int): Количество топ-кандидатов для NDCG и MAP.

    Returns:
        dict: Метрики F1, ROC-AUC, NDCG и MAP.
    """
    import numpy as np

    # Initialize lists to collect metrics
    y_true = []
    y_scores = []

    # Loop through test data
    for _, row in test_data.iterrows():
        person_skills = row["person_skills"]
        current_team_id = row["team_id"]

        # Form data for teams (including current team)
        teams = []
        for _, team_row in test_data.iterrows():
            teams.append({
                "team_id": int(team_row["team_id"]),
                "name": f"Team_{team_row['team_id']}",
                "skills": {"Member 1": team_row["team_skills"]},
                "required_roles": team_row["case_required_roles"]
            })

        # Form the request
        payload = {
            "person_skills": person_skills,
            "teams": teams,
            "role_filled_threshold": 0.5,
            "unfilled_role_weight": 1.5,
            "confidence_percentile": 0.9
        }

        # Send request
        response = requests.post(api_url, json=payload)
        if response.status_code == 200:
            recommendations = response.json()["recommended_teams"]
            
            # Create a mapping from team_id to similarity score
            team_scores = {rec["team_id"]: rec["similarity"] for rec in recommendations}
            
            # Collect scores for all teams
            for team in teams:
                team_id = team["team_id"]
                score = team_scores.get(team_id, 0)  # Get score if recommended, else 0
                y_scores.append(score)
                y_true.append(1 if team_id == current_team_id else 0)
        else:
            print(f"Request failed with status code: {response.status_code}")
            continue

    # Convert lists to numpy arrays
    y_true = np.array(y_true)
    y_scores = np.array(y_scores)

    # Calculate metrics
    from sklearn.metrics import roc_auc_score, f1_score, ndcg_score, average_precision_score

    # For binary classification metrics, we need predictions
    threshold = 0.5  # Adjust threshold as needed
    y_preds = (y_scores >= threshold).astype(int)

    # Calculate F1-Score
    f1 = f1_score(y_true, y_preds)

    # Calculate ROC-AUC
    roc_auc = roc_auc_score(y_true, y_scores)

    # Calculate NDCG
    ndcg = ndcg_score([y_true], [y_scores])

    # Calculate MAP
    map_score = average_precision_score(y_true, y_scores)

    # NDCG and MAP for Top-N
    top_n = 5
    sorted_indices = np.argsort(-y_scores)
    y_true_top_n = y_true[sorted_indices][:top_n]
    y_scores_top_n = y_scores[sorted_indices][:top_n]

    ndcg_top_n = ndcg_score([y_true_top_n], [y_scores_top_n])
    map_top_n = average_precision_score(y_true_top_n, y_scores_top_n)

    # Print metrics
    print({
        "F1-Score": f1,
        "ROC-AUC": roc_auc,
        "NDCG": ndcg,
        "MAP": map_score,
        "NDCG (Top-N)": ndcg_top_n,
        "MAP (Top-N)": map_top_n
    })


In [240]:
# Загрузка тестовых данных
test_data = pd.read_csv("test_dataset_skills.csv")
test_data

import ast

def safe_eval(x):
    if isinstance(x, str):
        try:
            # Пробуем преобразовать строку в список
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            # Если преобразование не удалось, пытаемся разделить строку по запятой
            return [item.strip() for item in x.split(",") if item.strip()]
    elif isinstance(x, list):
        # Если это уже список, возвращаем его
        return x
    return []  # Для других типов возвращаем пустой список


test_data["person_skills"] = test_data["person_skills"].apply(safe_eval)
test_data["team_skills"] = test_data["team_skills"].apply(safe_eval)
test_data["case_required_roles"] = test_data["case_required_roles"].apply(safe_eval)

# Тестирование и расчет метрик
metrics = test_api_and_evaluate(API_URL, test_data, model)
print("Metrics:")
print(metrics)

{'F1-Score': 0.011316533982498848, 'ROC-AUC': 0.4906238722675846, 'NDCG': 0.5397075147119422, 'MAP': 0.008435330448222002, 'NDCG (Top-N)': 0.0, 'MAP (Top-N)': -0.0}
Metrics:
None


/Users/riter/anaconda3/envs/dpp-recs/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [ ]:
{'F1-Score': 0.4353982300884956, 'ROC-AUC': 0.12751123663445163, 'NDCG': 0.5061839028142795, 'MAP': 0.021023298194017503, 'NDCG (Top-N)': 0.0, 'MAP (Top-N)': -0.0}


In [236]:
test_data.head()

,team_id,person_skills,team_skills,case_required_roles,target
0,54,"[Canva, Linux, Python]","[Git, Linux, Canva, Управление проектами, Pyth...","[ML engineer, Python Backend, DevOps, CV engin...",1
1,290,[Canva],[Canva],"[ML engineer, Дизайнер, Python Backend, CV eng...",1
2,28,[Управление проектами],"[DevOps, Linux, Умение работать с API, Back-en...","[ML engineer, Python Backend, Дизайнер, Аналит...",0
3,130,[Управление проектами],"[Jupyter, Computer Vision, Canva, Управление п...","[Дизайнер, Аналитик, Frontend, ML engineer, Py...",1
4,139,"[Python, SQL]","[DevOps, Git, Data Science, Linux, Умение рабо...","[Go Backend, ML engineer, DevOps, Тестировщик,...",1
